In [2]:
import sys
import numpy as np
sys.path.append("/home/kalic/Desktop/AI_P/CyberBattleSim")
from cyberbattle.simulation import model
from cyberbattle.samples.topology_nodeinfo import topology_and_nodeinfo_env
import gym
import cyberbattle.agents.baseline.agent_wrapper as w
import logging
import cyberbattle.agents.baseline.learner as learner
import cyberbattle.agents.baseline.plotting as p
import cyberbattle.agents.baseline.agent_randomcredlookup as rca
import cyberbattle.agents.baseline.agent_tabularqlearning as tqa
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.agents.baseline.agent_wrapper import Verbosity

Identifiers(properties=['CTFFLAG:LeakedCustomerData', 'CTFFLAG:LeakedCustomerData2', 'CTFFLAG:Readme.txt-Discover secret data', 'CTFFLAG:VMPRIVATEINFO', 'GitHub', 'MySql', 'SasUrlInCommit', 'SharepointLeakingPassword', 'Ubuntu', 'nginx/1.10.3'], ports=['GIT', 'HTTPS', 'MySQL', 'PING', 'SSH', 'SSH-key', 'su'], local_vulnerabilities=['CredScan-HomeDirectory', 'CredScanBashHistory', 'SearchEdgeHistory'], remote_vulnerabilities=['AccessDataWithSASToken', 'CredScanGitHistory', 'ListAzureResources', 'NavigateWebDirectory', 'NavigateWebDirectoryFurther', 'ScanPageContent', 'ScanPageSource', 'ScanSharepointParentDirectory'])


In [3]:
import sys
import numpy as np
sys.path.append("/home/kalic/Desktop/AI_P/CyberBattleSim")
import cyberbattle.agents.baseline.plotting as pptxy
import pickle

In [4]:
import gym
import cyberbattle.agents.baseline.agent_wrapper as w
# Load the Gym environment
# Papermill notebook parameters

#############
# gymid = 'CyberBattleTiny-v0'
#############
gymid = "CyberBattleToyCtf-v0"
env_size = None
iteration_count = 1500
training_episode_count = 20
eval_episode_count = 10
maximum_node_count = 12
maximum_total_credentials = 10
#############
# gymid = "CyberBattleChain-v0"
# env_size = 10
# iteration_count = 9000
# training_episode_count = 50
# eval_episode_count = 5
# maximum_node_count = 22
# maximum_total_credentials = 22
if env_size:
    gym_env = gym.make(gymid, size=env_size)
else:
    gym_env = gym.make(gymid)

ep = w.EnvironmentBounds.of_identifiers(
    maximum_node_count=maximum_node_count,
    maximum_total_credentials=maximum_total_credentials,
    identifiers=gym_env.identifiers
)


Identifiers(properties=['CTFFLAG:LeakedCustomerData', 'CTFFLAG:LeakedCustomerData2', 'CTFFLAG:Readme.txt-Discover secret data', 'CTFFLAG:VMPRIVATEINFO', 'GitHub', 'MySql', 'SasUrlInCommit', 'SharepointLeakingPassword', 'Ubuntu', 'nginx/1.10.3'], ports=['GIT', 'HTTPS', 'MySQL', 'PING', 'SSH', 'SSH-key', 'su'], local_vulnerabilities=['CredScan-HomeDirectory', 'CredScanBashHistory', 'SearchEdgeHistory'], remote_vulnerabilities=['AccessDataWithSASToken', 'CredScanGitHistory', 'ListAzureResources', 'NavigateWebDirectory', 'NavigateWebDirectoryFurther', 'ScanPageContent', 'ScanPageSource', 'ScanSharepointParentDirectory'])


In [10]:
with open('toy_paper_results/random_run_yuan.pkl','rb') as file:
    random_run_yuan = pickle.load(file)
with open('toy_paper_results/dql_run_yuan.pkl','rb') as file:
    dql_run_yuan = pickle.load(file)
with open('toy_paper_results/toy_0104_after_10_test.pkl','rb') as file:
    toy_my_dql_run_after_40_with_special_epsilon = pickle.load(file)

In [12]:
contenders = [
    # credlookup_run,
    # tabularq_run,
    # dql_run,
    # dql_exploit_run
    #load_m2,
    # random_run_1,
    #load_m3,
    random_run_yuan,
    
    # toy_my_dql_run_after_yuan,
    # toy_my_dql_run_after_40,
    toy_my_dql_run_after_40_with_special_epsilon,
    dql_run_yuan


    # load_m4,

    # load_m8,
]
pptxy.plot_episodes_length(contenders)
pptxy.plot_averaged_cummulative_rewards(
    title=f'Agent Benchmark top contenders\n'
    f'max_nodes:{ep.maximum_node_count}\n',
    all_runs=contenders)

In [2]:
env_size = None
iteration_count = 1500
training_episode_count = 20
eval_episode_count = 10
maximum_node_count = 12
maximum_total_credentials = 10

In [3]:
gymid = "multoycft_env-v0"
# size_N, size_NGN, dis_type, network_cc, connect_percent
gym_env = gym.make(gymid, size_N = 10, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.5)

In [7]:
print(ep)

EnvironmentBounds(maximum_total_credentials=10, maximum_node_count=12, maximum_discoverable_credentials_per_action=10, port_count=7, property_count=10, local_attacks_count=3, remote_attacks_count=8)


In [6]:
# Evaluate a Tabular Q-learning agent
tabularq_run = learner.epsilon_greedy_search(
    gym_env,
    ep,
    learner=tqa.QTabularLearner(
        ep,
        gamma=0.015, learning_rate=0.01, exploit_percentile=100),
    episode_count=1,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.01,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="Tabular Q-learning"
)

###### Tabular Q-learning
Learning with: episode_count=1,iteration_count=1500,ϵ=0.9,ϵ_min=0.01, ϵ_expdecay=5000,γ=0.015,learning_rate=0.01,Q%=100
  Episode 1 stopped at t=1500 loss={loss_string}
simulation ended


In [4]:
ep = w.EnvironmentBounds.of_identifiers(
    maximum_node_count=maximum_node_count,
    maximum_total_credentials=maximum_total_credentials,
    identifiers=gym_env.identifiers
)

In [5]:
# Evaluate the Deep Q-learning agent
my_dql_run1 = learner.epsilon_greedy_search(
    cyberbattle_gym_env=gym_env,
    environment_properties=ep,
    learner=dqla.DeepQLearnerPolicy(
        ep=ep,
        gamma=0.015,
        replay_memory_size=10000,
        target_update=10,
        batch_size=512,
        # torch default learning rate is 1e-2
        # a large value helps converge in less episodes
        learning_rate=0.01
    ),
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

###### DQL
Learning with: episode_count=20,iteration_count=1500,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10


/home/kalic/Desktop/AI_P/CyberBattleSim/cyberbattle/agents/baseline/agent_dql.py:382: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  state_batch = torch.tensor(states_to_consider).to(device)


  Episode 1 stopped at t=1500 
  Episode 2 stopped at t=1500 
  Episode 3 stopped at t=1500 
  Episode 4 stopped at t=1500 
  Episode 5 stopped at t=1500 
  Episode 6 stopped at t=1500 
  Episode 7 stopped at t=1500 
  Episode 8 stopped at t=1500 
  Episode 9 stopped at t=1500 
  Episode 10 stopped at t=1500 
  Episode 11 stopped at t=1500 
  Episode 12 stopped at t=1500 
  Episode 13 stopped at t=1500 
  Episode 14 stopped at t=1500 
  Episode 15 stopped at t=1500 
  Episode 16 stopped at t=1500 
  Episode 17 stopped at t=1500 
  Episode 18 stopped at t=1500 
  Episode 19 stopped at t=1500 
  Episode 20 stopped at t=1500 
simulation ended


In [7]:
print(my_dql_run1['all_episodes_rewards'])

[[36.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0

In [8]:
my_steps_N = []
my_rewards_N = []

In [18]:
N = 5
for i in range(N):
    # Evaluate the Deep Q-learning agent
    gymid = "multoycft_env-v0"
    gym_env = gym.make(gymid, size_N = 10, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.5)
    my_dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=my_dql_run1['learner'],
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )

    my_steps_N.append(my_dql_run['all_steps'])
    my_rewards_N.append(my_dql_run['all_episodes_rewards'])
    #print(i, my_dql_run['all_steps'], my_dql_run['all_episodes_rewards'])
    my_dql_run1 = my_dql_run


###### DQL
Learning with: episode_count=20,iteration_count=1500,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10


  Episode 1 stopped at t=1500 
  Episode 2 stopped at t=1500 
  Episode 3 stopped at t=1500 
  Episode 4 stopped at t=1500 
  Episode 5 stopped at t=1500 
  Episode 6 stopped at t=1500 
  Episode 7 stopped at t=1500 
  Episode 8 stopped at t=1500 
  Episode 9 stopped at t=1500 
  Episode 10 stopped at t=1500 
  Episode 11 stopped at t=1500 
  Episode 12 stopped at t=1500 
  Episode 13 stopped at t=1500 
  Episode 14 stopped at t=1500 
  Episode 15 stopped at t=1500 
  Episode 16 stopped at t=1500 
  Episode 17 stopped at t=1500 
  Episode 18 stopped at t=1500 
  Episode 19 stopped at t=1500 
  Episode 20 stopped at t=1500 
simulation ended
###### DQL
Learning with: episode_count=20,iteration_count=1500,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=1500 
  Episode 2 stopped at t=1500 
  Episode 3 stopped at t=1500 
  Episode 4 stopped at t=1500 
  Episode 5 stopped at t=1500 
  Episode 6 stopped at t=1500 
  E

In [22]:
gymid2 = "CyberBattleToyCtf-v0"
if env_size:
    gym_env = gym.make(gymid, size=env_size)
else:
    gym_env = gym.make(gymid)

ep = w.EnvironmentBounds.of_identifiers(
    maximum_node_count=maximum_node_count,
    maximum_total_credentials=maximum_total_credentials,
    identifiers=gym_env.identifiers
)

In [23]:
# Evaluate the Deep Q-learning agent
dql_run = learner.epsilon_greedy_search(
    cyberbattle_gym_env=gym_env,
    environment_properties=ep,
    learner=my_dql_run['learner'],
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

###### DQL
Learning with: episode_count=20,iteration_count=1500,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=1500 
  Episode 2 stopped at t=1500 
  Episode 3 stopped at t=1500 
  Episode 4 stopped at t=1500 
  Episode 5 stopped at t=1500 
  Episode 6 stopped at t=1500 
  Episode 7 stopped at t=1500 
  Episode 8 stopped at t=1500 
  Episode 9 stopped at t=1500 
  Episode 10 stopped at t=1500 
  Episode 11 stopped at t=1500 
  Episode 12 stopped at t=1500 
  Episode 13 stopped at t=1500 
  Episode 14 stopped at t=1500 
  Episode 15 stopped at t=1500 
  Episode 16 stopped at t=1500 
  Episode 17 stopped at t=1500 
  Episode 18 stopped at t=1500 
  Episode 19 stopped at t=1500 
  Episode 20 stopped at t=1500 
simulation ended


In [10]:
import pickle

In [19]:
with open('toy_my_dql_run_round_10.pkl','wb') as file:
    pickle.dump(my_dql_run, file)

In [16]:
np.savetxt('toy_my_dql_run_round_5_my_steps_N.txt', my_steps_N, fmt='%d', delimiter=",")


In [20]:
with open('toy_my_dql_run_round_10_DQN_steps_N.txt', 'w') as outfile:
    for slice_2d in my_rewards_N:
        np.savetxt(outfile, slice_2d, fmt = '%f', delimiter = ',')
# np.savetxt('toy_my_dql_run_round_5_DQN_steps_N.txt', my_rewards_N, fmt='%d', delimiter=",")

In [10]:
print(my_dql_run1['all_steps'], my_dql_run1['all_episodes_rewards'])

[487, 365, 460, 281, 459, 194, 341, 468, 276, 310] [284.0, 361.0, 361.0, 361.0, 367.0, 367.0, 361.0, 361.0, 361.0, 361.0]


In [24]:
import cyberbattle.agents.baseline.plotting as p

In [32]:
with open('toy_random_run.pkl','rb') as file:
    load_m = pickle.load(file)

In [36]:
with open('toy_tabularq_run.pkl','rb') as file:
    load_m2 = pickle.load(file)

In [41]:
import cyberbattle.agents.baseline.plotting as p

In [42]:
# Compare and plot results for all the agents
all_runs = [
    # random_run,
    # credlookup_run,
    # tabularq_run,
    # tabularq_exploit_run,
    dql_run,
    load_m,
    load_m2
    # dql_exploit_run
]

# Plot averaged cumulative rewards for DQL vs Random vs DQL-Exploit
themodel = dqla.CyberBattleStateActionModel(ep)
p.plot_averaged_cummulative_rewards(
    all_runs=all_runs,
    title=f'Benchmark -- max_nodes={ep.maximum_node_count}, episodes={eval_episode_count},\n'
    f'State: {[f.name() for f in themodel.state_space.feature_selection]} '
    f'({len(themodel.state_space.feature_selection)}\n'
    f"Action: abstract_action ({themodel.action_space.flat_size()})",show=True)

In [ ]:
dql_run